# Setup

In [1]:
# Import packages
import pandas as pd
import numpy as np
import sqlite3 as sql
import datetime

In [2]:
# Create a connection to our local SQLLite database
db_file = 'sqlite_db_pythonsqlite.db'
connection = sql.connect(db_file)

In [3]:
# Query our database
query_fac = 'SELECT * FROM Facilities'
df_fac = pd.read_sql_query(query_fac, connection)

query_book = 'SELECT * FROM Bookings'
df_book = pd.read_sql_query(query_book, connection)

query_mem = 'SELECT * FROM Members'
df_mem = pd.read_sql_query(query_mem, connection)

In [4]:
df_fac

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [5]:
df_book.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [6]:
df_mem.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


# Q10

Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [7]:
## Merge the facilities and bookings dataframes based on facid
# Check the shape
df_fac_book = df_fac.merge(df_book, left_on='facid', right_on='facid')
print(df_fac_book.shape)
df_fac_book.head()

(4043, 10)


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,bookid,memid,starttime,slots
0,0,Tennis Court 1,5.0,25.0,10000,200,6,2,2012-07-04 09:00:00,3
1,0,Tennis Court 1,5.0,25.0,10000,200,7,2,2012-07-04 15:00:00,3
2,0,Tennis Court 1,5.0,25.0,10000,200,26,0,2012-07-06 08:00:00,3
3,0,Tennis Court 1,5.0,25.0,10000,200,27,0,2012-07-06 14:00:00,3
4,0,Tennis Court 1,5.0,25.0,10000,200,28,2,2012-07-06 15:30:00,3


In [8]:
# Now let's add a column for revenue per booking (a memid of 0 indicates a guest)
# df_fac_book['revenue'] = np.where(df_fac_book['memid']==0,df_fac_book['guestcost'], df_fac_book['membercost'] ) * df_fac_book['slots']

# Let's add a new column to the dataframe to calculate revenue per booking
# A member of 0 indicates a guest and we use the guest fee; otherwise we use the member fee
# We'll use a lambda function
df_fac_book['revenue'] = df_fac_book.apply(lambda row: row['slots']*(row['guestcost'] if row['memid']==0 else row['membercost']), axis=1)

# Now we'll group by facid (and name so we preserve the name) summing the revenue
df_revenue_per_facid = df_fac_book.groupby(by=['facid','name']).sum()

# Reset the index
df_revenue_per_facid = df_revenue_per_facid.reset_index()

In [9]:
df_revenue_per_facid[df_revenue_per_facid['revenue']<1000][['name','revenue']].sort_values(by='revenue')

,name,revenue
3,Table Tennis,180.0
7,Snooker Table,240.0
8,Pool Table,270.0


# Q11 
Produce a report of members and who recommended them in alphabetic surname,firstname order.



In [10]:
# Let's drop the Guest row since guests aren't members
df_mem = df_mem.drop([0])

In [11]:
# Let's add a column with membername formatted the way we like it
df_mem['membername']=df_mem['surname'] + ", " + df_mem['firstname']

In [12]:
# Iterate over the rows, doing a lookup on the index to grab the membername based on the value of recommendedby 
# Put N/A in for null values

for index, row in df_mem.iterrows():
    df_mem.at[index,'rec_by_name']='N/A' if row['recommendedby']=="" else df_mem['membername'][int(row['recommendedby'])] 

In [13]:
df_mem[['membername','rec_by_name']].sort_values(by='membername')

,membername,rec_by_name
15,"Bader, Florence","Stibbons, Ponder"
12,"Baker, Anne","Stibbons, Ponder"
16,"Baker, Timothy","Farrell, Jemima"
8,"Boothe, Tim","Rownam, Tim"
5,"Butters, Gerald","Smith, Darren"
20,"Coplin, Joan","Baker, Timothy"
29,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
24,"Farrell, David",N/A
13,"Farrell, Jemima",N/A


# Q11
Find the facilities with their usage by member, but not guests


In [14]:
# Grab names of facilities and members, convert them to appropriate dataframe with appropriately name columns
# so we can merge them with our grouped dataframe
fac_names = pd.DataFrame(df_fac['name']).reset_index()
fac_names.columns=['facid','facname']

mem_names = pd.DataFrame(df_mem['membername']).reset_index()
mem_names.columns=['memid','membername']                        

# Let's get rid of guest data from the booking dataframe and group that dataframe
df_grouped = df_book[df_book['memid']!=0][['facid','memid','slots']].groupby(by=['facid','memid']).sum()

# Add a column to represent usage in hours
df_grouped['usage in hours'] = df_grouped['slots']/2 

# Let's reset the index and then join with facility names and member names
df_grouped = df_grouped.reset_index()

# Merge with the member names
df_output = df_grouped.merge(mem_names, left_on = 'memid', right_on='memid')
print(df_grouped.shape, df_output.shape)


(202, 4) (187, 5)


Notice that after we do the merge with member name, we have fewer rows.  This is because 
the bookings table contains invalid member ids that are not in the member table.  Because
merge by default does an inner join, these rows are dropped.  This is ok as there is nothing
else we could do with this data

In [15]:
# Merge with the facility names
df_output = df_output.merge(fac_names, left_on = 'facid', right_on = 'facid')

In [16]:
df_output.shape

(187, 6)

Still the same number of rows, so all the facility ids were valid.  

In [17]:
# Print out our result
df_output[['facname','membername','usage in hours']].sort_values(['facname','membername'])

,facname,membername,usage in hours
53,Badminton Court,"Bader, Florence",13.5
50,Badminton Court,"Baker, Anne",15.0
54,Badminton Court,"Baker, Timothy",10.5
46,Badminton Court,"Boothe, Tim",18.0
43,Badminton Court,"Butters, Gerald",31.5
...,...,...,...
33,Tennis Court 2,"Smith, Jack",1.5
21,Tennis Court 2,"Smith, Tracy",3.0
28,Tennis Court 2,"Stibbons, Ponder",48.0
25,Tennis Court 2,"Tracy, Burton",4.5


# Q12
Find the facilities usage by month, but not guests

In [18]:
# Convert starttime to a datetime object
df_book['starttime'] = pd.to_datetime(df_book['starttime'])

# Add a column to the booking dataframe for the month
df_book['month'] = pd.DatetimeIndex(df_book['starttime']).month

# Let's get rid of guest data from the booking dataframe and group that dataframe
df_grouped = df_book[df_book['memid']!=0][['facid','month','slots']].groupby(by=['facid','month']).sum()

# Add a column to represent usage in hours
df_grouped['usage in hours'] = df_grouped['slots']/2 

In [19]:
# Let's reset the index and then join with facility names
df_grouped = df_grouped.reset_index()

# Now merge with facility names
df_output = df_grouped.merge(fac_names, left_on = 'facid', right_on = 'facid')

In [20]:
# Print out our result
df_output[['facname','month','usage in hours']].sort_values(['facname','month'])

,facname,month,usage in hours
6,Badminton Court,7,82.5
7,Badminton Court,8,207.0
8,Badminton Court,9,253.5
12,Massage Room 1,7,83.0
13,Massage Room 1,8,158.0
14,Massage Room 1,9,201.0
15,Massage Room 2,7,4.0
16,Massage Room 2,8,9.0
17,Massage Room 2,9,14.0
24,Pool Table,7,55.0
